In [7]:
import folium
from main import main_train
from viewer import get_position, view

In [8]:
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
# プロット
for p in get_position().values():
    folium.Marker(location=p).add_to(map) 
map

In [9]:
from get_data import get_spots, get_time
pos = get_position()
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
d = get_time("嵐山")
spots = get_spots()
lines = []
for i in range(25):
    for j in range(25):
        if i != j and d[i][j] < 10000:
            lines.append({"from": pos[spots[i]], "to": pos[spots[j]]})
# プロット
for p in pos.values():
    folium.Marker(location=p).add_to(map) 
for line in lines:
    folium.PolyLine(
        locations=[line["from"], line["to"]]
    ).add_to(map)
map.save('result/round_map.html')
map

In [10]:
unique_results = main_train(num_loop=96, train_loop=10_000_000) #10_000_000

2h 2m 22s


In [11]:
route = unique_results[0][1]
view(route)


In [13]:
import pickle
def save(result, country):
    file_name = "result/{}.pkl".format(country)
    with open(file_name, "wb") as f:
        pickle.dump(result, f)

save(unique_results,"中国嵐山2")

In [15]:
country = "中国"
for i in [0, 2, 4,6]:
    if i < len(unique_results):
        route = unique_results[i][1]
        view(route).save('result/{}_{}_add.html'.format(country, i + 1))